In [1]:
import os
import requests
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model, model_selection

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader

import torchvision
from torchvision import transforms
from torchvision.utils import make_grid
from torchvision.models import resnet18
from copy import deepcopy
import torch.nn.functional as F
from sklearn import linear_model, model_selection
from sklearn.metrics import make_scorer, accuracy_score
from tqdm.notebook import tqdm

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on device:", DEVICE.upper())

from typing import Callable

# manual random seed is used for dataset partitioning
# to ensure reproducible results across runs
RNG = torch.Generator().manual_seed(42)

Running on device: CPU


In [2]:
torch.cuda.is_available()

False

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

import os
from distutils.dir_util import copy_tree, remove_tree

In [4]:
ls

 Volume in drive C is Windows
 Volume Serial Number is BA18-FA02

 Directory of C:\Indiana University Bloomington\Machine_Unlearning

12/07/2023  04:09 PM    <DIR>          .
11/07/2023  03:51 PM    <DIR>          ..
12/07/2023  03:27 PM    <DIR>          .ipynb_checkpoints
11/30/2023  07:47 PM         1,412,207 1.pdf
11/30/2023  07:47 PM           631,462 2.pdf
11/30/2023  07:47 PM         1,185,882 3.pdf
11/30/2023  07:47 PM         1,255,077 4.pdf
11/30/2023  07:47 PM        13,992,663 5.pdf
11/30/2023  07:47 PM           912,777 6.pdf
11/30/2023  07:48 PM         1,281,178 7.pdf
12/07/2023  03:37 PM    <DIR>          Alzheimer_data
12/07/2023  04:09 PM            45,272 alzimers-model-train.ipynb
12/07/2023  03:57 PM    <DIR>          dataset
12/04/2023  09:58 PM            36,093 evaluation-baseline-model.ipynb
12/06/2023  07:03 PM           142,236 evaluation-reset-last-layers-baseline.ipynb
12/06/2023  09:20 AM           123,910 evaluation-retrain-model.ipynb
12/06/2023  11:45 A

In [5]:
alzimer_dataset_directory = './Alzheimer_data/'

In [6]:
base_dir = "./Alzheimer_data/"
root_dir = "./"
test_dir = base_dir + "test/"
train_dir = base_dir + "train/"
work_dir = root_dir + "dataset/"

if os.path.exists(work_dir):
    remove_tree(work_dir)
    

os.mkdir(work_dir)
copy_tree(train_dir, work_dir)
copy_tree(test_dir, work_dir)
print("Working Directory Contents:", os.listdir(work_dir))

Working Directory Contents: ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']


In [7]:
CLASSES = [ 'NonDemented',
            'VeryMildDemented',
            'MildDemented',
            'ModerateDemented']

IMG_SIZE = 150
IMAGE_SIZE = [150, 150]
DIM = (IMG_SIZE, IMG_SIZE)

In [8]:
ZOOM = [.99, 1.01]
BRIGHT_RANGE = [0.8, 1.2]
HORZ_FLIP = True
FILL_MODE = "constant"
DATA_FORMAT = "channels_last"

train_dr = IDG(rescale = 1./255, brightness_range=BRIGHT_RANGE, zoom_range=ZOOM, data_format=DATA_FORMAT, fill_mode=FILL_MODE, horizontal_flip=HORZ_FLIP)

train_data_gen = train_dr.flow_from_directory(directory=train_dir, target_size=DIM, batch_size=6500, shuffle=False)

Found 5121 images belonging to 4 classes.


In [9]:
train_data, train_labels = train_data_gen.next()

In [10]:
print(train_data.shape, train_labels.shape)

(5121, 150, 150, 3) (5121, 4)


In [11]:
train_labels

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]], dtype=float32)

## Getting the train data and augmenting it

In [12]:
train_dr = IDG(rescale = 1./255, brightness_range=BRIGHT_RANGE, zoom_range=ZOOM, data_format=DATA_FORMAT, fill_mode=FILL_MODE, horizontal_flip=HORZ_FLIP)

train_data_gen = train_dr.flow_from_directory(directory=train_dir, target_size=DIM, batch_size=6500, shuffle=False)

train_data, train_labels = train_data_gen.next()
#Performing over-sampling of the data, since the classes are imbalanced
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)

train_data, train_labels = sm.fit_resample(train_data.reshape(-1, IMG_SIZE * IMG_SIZE * 3), train_labels)

train_data = train_data.reshape(-1, IMG_SIZE, IMG_SIZE, 3)

print(train_data.shape, train_labels.shape)

(10240, 150, 150, 3) (10240, 4)


## Getting the test data and augmenting it

In [13]:
test_dr = IDG(rescale = 1./255, brightness_range=BRIGHT_RANGE, zoom_range=ZOOM, data_format=DATA_FORMAT, fill_mode=FILL_MODE, horizontal_flip=HORZ_FLIP)

test_data_gen = test_dr.flow_from_directory(directory=test_dir, target_size=DIM, batch_size=6500, shuffle=False)

test_data, test_labels = test_data_gen.next()

from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)

test_data, test_labels = sm.fit_resample(test_data.reshape(-1, IMG_SIZE * IMG_SIZE * 3), test_labels)

test_data = test_data.reshape(-1, IMG_SIZE, IMG_SIZE, 3)

print(test_data.shape, test_labels.shape)

Found 1279 images belonging to 4 classes.
(2560, 150, 150, 3) (2560, 4)


In [14]:
# from sklearn.model_selection import train_test_split

# train_data, test_data, train_labels, test_labels = train_test_split(train_data, train_labels, test_size = 0.2, random_state=42)
# train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size = 0.2, random_state=42)

## I have defined the model in the below few steps - Please use this as your new model

In [15]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
inception_model = InceptionV3(input_shape=(150, 150, 3), include_top=False, weights="imagenet")

In [16]:
for layer in inception_model.layers:
    layer.trainable=False

In [17]:
import tensorflow_addons as tfa
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import SeparableConv2D, BatchNormalization, GlobalAveragePooling2D

custom_inception_model = Sequential([
        inception_model,
        Dropout(0.5),
        GlobalAveragePooling2D(),
        Flatten(),
        BatchNormalization(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        BatchNormalization(),
        Dense(4, activation='softmax')        
    ], name = "inception_cnn_model")

C:\Users\ranka\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
C:\Users\ranka\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.15.0 and is not supported. 
Some things might work, some things might not.
If you were to encoun

In [18]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),  
      tf.keras.metrics.AUC(name='auc')
]

# CALLBACKS = rop_callback
    
custom_inception_model.compile(optimizer='adam',
                              loss=tf.losses.CategoricalCrossentropy(),
                              metrics=METRICS)

custom_inception_model.summary()


Model: "inception_cnn_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 3, 3, 2048)        21802784  
                                                                 
 dropout (Dropout)           (None, 3, 3, 2048)        0         
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 batch_normalization_94 (Ba  (None, 2048)              8192      
 tchNormalization)                                               
                                                                 
 dense (Dense)               (None, 512)      

In [20]:
#Fit the training data to the model and validate it using the validation data
EPOCHS = 20

history = custom_inception_model.fit(train_data, train_labels, validation_data=(test_data, test_labels), epochs=EPOCHS)

Epoch 1/20

320/320 [==============================] - 62s 169ms/step - loss: 1.2763 - accuracy: 0.7595 - precision: 0.5328 - recall: 0.3105 - auc: 0.7248 - val_loss: 1.1701 - val_accuracy: 0.7645 - val_precision: 0.5661 - val_recall: 0.2477 - val_auc: 0.7327
Epoch 2/20
320/320 [==============================] - 60s 187ms/step - loss: 0.8776 - accuracy: 0.8153 - precision: 0.7304 - recall: 0.4138 - auc: 0.8546 - val_loss: 1.2317 - val_accuracy: 0.7649 - val_precision: 0.5574 - val_recall: 0.2902 - val_auc: 0.7500
Epoch 3/20
320/320 [==============================] - 78s 245ms/step - loss: 0.7773 - accuracy: 0.8331 - precision: 0.7786 - recall: 0.4644 - auc: 0.8877 - val_loss: 1.1053 - val_accuracy: 0.7917 - val_precision: 0.6515 - val_recall: 0.3586 - val_auc: 0.7988
Epoch 4/20
320/320 [==============================] - 83s 260ms/step - loss: 0.7122 - accuracy: 0.8440 - precision: 0.7879 - recall: 0.5146 - auc: 0.9063 - val_loss: 1.4485 - val_accuracy: 0.7711 - val_precision: 0.5902 - 

In [23]:
custom_inception_model.save_weights('origional_model_weights.h5')

# You can also save the entire model (architecture + weights + optimizer state) to a single file
custom_inception_model.save('all_origional_model.h5')

C:\Users\ranka\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [21]:
retrain_inception_model = Sequential([
        inception_model,
        Dropout(0.5),
        GlobalAveragePooling2D(),
        Flatten(),
        BatchNormalization(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        BatchNormalization(),
        Dense(4, activation='softmax')        
    ], name = "retrain_inception_cnn_model")

METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),  
      tf.keras.metrics.AUC(name='auc')
]

# CALLBACKS = rop_callback
    
retrain_inception_model.compile(optimizer='adam',
                              loss=tf.losses.CategoricalCrossentropy(),
                              metrics=METRICS)

retrain_inception_model.summary()

Model: "retrain_inception_cnn_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 3, 3, 2048)        21802784  
                                                                 
 dropout_5 (Dropout)         (None, 3, 3, 2048)        0         
                                                                 
 global_average_pooling2d_1  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 flatten_1 (Flatten)         (None, 2048)              0         
                                                                 
 batch_normalization_99 (Ba  (None, 2048)              8192      
 tchNormalization)                                               
                                                                 
 dense_5 (Dense)             (None, 512

In [22]:
train_data.shape

(10240, 150, 150, 3)

In [24]:
total_indices = len(train_data)
print(total_indices)

num_indices_to_select = int(0.1 * total_indices)

# Generate a random sample of indices
random_indices = np.random.choice(total_indices, size=num_indices_to_select, replace=False)

10240


In [26]:
len(random_indices)

1024

In [27]:
random_indices

array([6184, 2095, 9759, ...,  161, 9376, 8479])

In [28]:
file_path = 'forget_indices.txt'

# Save the random indices to the text file
np.savetxt(file_path, random_indices, fmt='%d')

In [29]:
forget_data, forget_label = train_data[random_indices], train_labels[random_indices]

In [30]:
len(forget_data)

1024

In [31]:
mask = np.isin(np.arange(len(train_data)), random_indices, invert=True)

# Use the boolean mask to obtain the subset of train_data
retain_data, retain_labels = train_data[mask], train_labels[mask]

In [32]:
len(retain_data)

9216

In [33]:
len(train_data)

10240

In [ ]:
EPOCHS = 20

history = retrain_inception_model.fit(retain_data, retain_labels, validation_data=(test_data, test_labels), epochs=EPOCHS)

Epoch 1/20
288/288 [==============================] - 63s 184ms/step - loss: 1.3122 - accuracy: 0.7528 - precision: 0.5095 - recall: 0.2956 - auc: 0.7085 - val_loss: 1.0970 - val_accuracy: 0.7781 - val_precision: 0.7835 - val_recall: 0.1555 - val_auc: 0.7575
Epoch 2/20
288/288 [==============================] - 53s 183ms/step - loss: 0.9046 - accuracy: 0.8114 - precision: 0.7159 - recall: 0.4073 - auc: 0.8473 - val_loss: 1.2233 - val_accuracy: 0.7808 - val_precision: 0.6986 - val_recall: 0.2164 - val_auc: 0.7527
Epoch 3/20
288/288 [==============================] - 54s 188ms/step - loss: 0.8094 - accuracy: 0.8246 - precision: 0.7519 - recall: 0.4452 - auc: 0.8778 - val_loss: 1.2635 - val_accuracy: 0.7772 - val_precision: 0.6233 - val_recall: 0.2754 - val_auc: 0.7465
Epoch 4/20
288/288 [==============================] - 63s 220ms/step - loss: 0.7609 - accuracy: 0.8371 - precision: 0.7672 - recall: 0.5000 - auc: 0.8937 - val_loss: 1.2452 - val_accuracy: 0.7826 - val_precision: 0.6135 - v

In [ ]:
retrain_inception_model.save_weights('retrain_model_weights.h5')

# You can also save the entire model (architecture + weights + optimizer state) to a single file
retrain_inception_model.save('all_retrain_model.h5')